In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import pandas as pd 
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

In [ ]:
"""
lable_studio标注数据转化成BIO格式

"""
import json

def json_to_bio(json_file, bio_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    output_data = []
    
    for item in data:
        text = item['data']['text']
        entities = item['annotations'][0]['result']
        bio_tags = get_bio_tags(text, entities)
        
        # Prepare data in the desired format
        output_item = {
            'id': item['id'],
            'text': text.split(),  # Split text into words
            'ner_tags': [label2id[label] for label in bio_tags]
        }
        output_data.append(output_item)
    
    with open(bio_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)

def get_bio_tags(text, entities):
    words = text.split()
    bio_tags = ['O'] * len(words)  # Initialize with 'O' for each word
    
    for entity in entities:
        start = entity['value']['start']
        end = entity['value']['end']
        entity_type = entity['value']['labels'][0]
        
        for i in range(start, end):
            if i < len(words):
                bio_tags[i] = 'B-' + entity_type if i == start else 'I-' + entity_type
    
    return bio_tags

if __name__ == '__main__':
    label_list = ['O',"B-Gender","I-Gender","B-Age","I-Age", "B-Conditions","I-Conditions", 
                "B-Intervention","I-Intervention", "B-Outcome", "I-Outcome"]
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for i, label in enumerate(label_list)}
    input_from_labelstudio = "project-1-at-2024-07-04-17-24-6231169e.json"
    output_to_spanbase = 'work_project.json'
    json_to_bio(input_from_labelstudio, output_to_spanbase)


In [ ]:
from datasets import load_dataset
ner_datasets = load_dataset('json', data_files='work_project.json')

train_test_split = ner_datasets['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
valid_test_split = test_dataset.train_test_split(test_size=0.5, shuffle=True, seed=42)
valid_dataset = valid_test_split['train']
test_dataset = valid_test_split['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
ner_datasets = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})
print(ner_datasets)   #查看数据总体描述
print(ner_datasets["train"][0])  #查看数据样例 train数据集中的某一个样例
print(ner_datasets["train"].features) #查看不同数据集中键的说明
#label_list=ner_datasets["train"].features["ner_tags"].feature.names 

In [ ]:
yourdatasets=ner_datasets

#1.声明和指定一个分词器
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")  
#2.声明函数：处理tokens和ner_tags
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["text"], 
                                   max_length=350, 
                                   truncation=True, 
                                   is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples
# 3.处理
tokenized_datasets= yourdatasets.map(process_function, batched=True)
print(tokenized_datasets)
print(tokenized_datasets["train"][0])

In [ ]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained(
    "allenai/scibert_scivocab_uncased", 
    num_labels=len(label_list))
model.config.num_labels

seqeval = evaluate.load("seqeval")
print(seqeval)  #查看使用方法

import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, 
                             references=true_labels, 
                             mode="strict", 
                             scheme="IOB2")

    return { "f1": result["overall_f1"]}

In [ ]:
# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 优化后的 TrainingArguments
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,  # 根据 GPU 内存调整批量大小
    per_device_eval_batch_size=128,   # 根据 GPU 内存调整批量大小
    evaluation_strategy="epoch",     # 每个 epoch 评估一次
    save_strategy="epoch",           # 每个 epoch 保存一次
    metric_for_best_model="f1", 
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=5,              # 训练 1 次
    fp16=True,                       # 如果支持，使用混合精度训练
    dataloader_num_workers=4         # 增加数据加载线程数
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

trainer.train()


In [ ]:
from datasets import load_dataset
ner_datasets = load_dataset('json', data_files='EBM_NLP_2_Huggingface.json')

train_test_split = ner_datasets['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
valid_test_split = test_dataset.train_test_split(test_size=0.5, shuffle=True, seed=42)
valid_dataset = valid_test_split['train']
test_dataset = valid_test_split['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
ner_datasets = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})
print(ner_datasets)   #查看数据总体描述
print(ner_datasets["train"][0])  #查看数据样例 train数据集中的某一个样例
print(ner_datasets["train"].features) #查看不同数据集中键的说明
#label_list=ner_datasets["train"].features["ner_tags"].feature.names 
 


In [ ]:
yourdatasets=ner_datasets

#1.声明和指定一个分词器
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")  
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

#2.声明函数：处理tokens和ner_tags
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["text"], 
                                   max_length=350, 
                                   truncation=True, 
                                   is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["label"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples
# 3.处理
tokenized_datasets= yourdatasets.map(process_function, batched=True)
print(tokenized_datasets)
print(tokenized_datasets["train"][0])